# Capstone project for Coursera IBM Data Science

This will be used for the Coursera IBM Data Science Capstone Project

In [ ]:
# Needed for geocoder.
#!conda install -c conda-forge  geocoder --yes

In [ ]:
# Needed for folium
#!conda install -c conda-forge folium

In [ ]:
# On cognitiveclass.ai: Needed for pd.read_html() 
#!conda install lxml --yes

In [ ]:
import pandas as pd
import numpy as np

import requests # library to handle requests
#import random # library for random number generation

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# geo plotting library
import folium
import folium.plugins

# Handle json parsing
import json

# Need to sleep for rate limit to foursquare
from time import sleep

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
print('Libraries imported.')

# Table of Contents

1. Phase 1: Scrape and Transform
2. Phase 2: Apply Latitude and Longitude
3. Phase 3: Analyze and Map

# Phase 1: Scrape and Transform

### 1. Scrape postal codes and neighborhoods from Wikipedia
Out of all the tables, the first one with the text "Borough" has the data we need.

In [ ]:
postal_codes_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
postal_codes_raw = pd.read_html(io=postal_codes_url, match="Borough")[0]    # Grab the first table
postal_codes_raw.head()

### 2. Remove 'Not assigned' entries

In [ ]:
# Verify that All Neighbourhoods that are 'Not assigned' also do not have a Borough
q = postal_codes_raw[(postal_codes_raw['Neighbourhood'] == 'Not assigned') & (postal_codes_raw['Neighbourhood'] != postal_codes_raw['Borough'])]
q

In [ ]:
postal_codes_with_borough = postal_codes_raw[postal_codes_raw['Borough'] != 'Not assigned']
postal_codes_with_borough.head()

### 3. Consolidate by Postcode

In [ ]:
# Group by columns
keycolumns=['Postcode','Borough']
# Preserve the keys with a multiindex
mi = pd.MultiIndex.from_frame(postal_codes_with_borough[keycolumns])
# Create an independent dataframe with the key as the index and only the neighborhood as the column
postal_codes_indexed = postal_codes_with_borough\
                    .copy()\
                    .set_index(mi)\
                    .drop(columns=keycolumns)
# Concatenate the neighborhoods with a comma
postal_codes_clean = postal_codes_indexed\
                    .groupby(keycolumns)\
                    .aggregate(lambda x: x.str.cat(sep=','))
# Move the index back to columns
postal_codes_clean.reset_index(inplace=True,level='Borough')
postal_codes_clean.sort_values('Postcode',inplace=True)
postal_codes_clean.head()

### 4. And the answer is:

In [ ]:
postal_codes_clean.shape

# Phase 2: Attach geocoding

**Note:** The geocoder package refused to work. Looks like it depends on an API Key that costs money

### 1. Get the Geocoded Postal Codes

In [ ]:
#geocode_url = "https://cocl.us/Geospatial_data"
#geodata = pd.read_csv(geocode_url)
#geodata.head()

In [ ]:
# Supplied csv file, unavailable due to problems with code class
urlGeo_template='http://geogratis.gc.ca/services/geolocation/en/locate?q='
q = []
processed = 0
for p in postal_codes_clean.iterrows():
    postcode = p[0]
    url=urlGeo_template + postcode
    results = requests.get(url).json()
    if (len(results) > 0):
        location = results[0]['geometry']['coordinates']
        q.append([postcode, *location])
        #print(postcode, location)
    processed = processed + 1
print('Done. Located: ',processed)

In [ ]:
geodata = pd.DataFrame(q,columns=['Postal Code','Longitude','Latitude'])
#geodata.set_index('Postal Code', inplace=True)
geodata.head()

### 2. Attach geocoding to Neighbourhoods

In [ ]:
# Rename columns, make it an index for the join, sort it
geodata_clean =geodata\
                .rename(columns={'Postal Code': 'Postcode'})\
                .set_index('Postcode')\
                .sort_values('Postcode')
geodata_clean.head()

In [ ]:
postal_codes_final = postal_codes_clean.join(geodata_clean, how='left')
postal_codes_final.reset_index().head()

# Phase 3: Analyze and Map

In [ ]:
# @hide_cell
CLIENT_ID = '5FRFUXBNYR421XV1CZRBPDBTYFW0QVCMAXF5PASKCIGBU3I0'
CLIENT_SECRET= 'I22MHF1CYB5C44H2SRWF0SOQ1KA4DR1LJ0K2RVIYCYAGZQNL'
OAUTH_TOKEN="EILNUTET0HYQEQH155OYZ0MXVMARHZBVUZVJEQJKTULWIOUF" # User Auth Token

In [ ]:
VERSION = '20180323'
LIMIT=100
RADIUS=1000
INTENT='browse'
url_base = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}'
secret = [CLIENT_ID,CLIENT_SECRET]
#url_base = 'https://api.foursquare.com/v2/venues/search?&oauth_token={}'
#secret = [OAUTH_TOKEN]

url_template = (url_base + '&v={}&intent={}&limit={}&radius={}').format(
    *secret,
    VERSION,
    INTENT,
    LIMIT,
    RADIUS
    )
url_template = url_template + '&ll={},{}'    # &radius={} -- Get as much as will be supplied
url_template

In [ ]:
# Extract only the primary category, if more than one. If no categories return None.
def get_primary_category(categories):
    for c in categories:
        if c['primary']:
            return c['name']
    return None

In [ ]:
# Given description and lat, lon 
# Return the venue id, and associated categories
def get_venues_for(postcode, borough, lat, lon):
    sleep(.5) # personal allows 2 queries/sec.
    print(postcode, borough, lat, lon,' ',end='')
    url=url_template.format(lat, lon, 100)
    #print(url,' ', end='')
    results = requests.get(url).json()
    response = results['response']
    retval = None
    venues = None
    if 'venues' in response:
        venues = response['venues']
        retval = {'Postcode': postcode, 'Borough': borough, 'Venues': venues}
        print('Found', len(venues))
    else:
        print('No venues returned')
    return retval

In [ ]:
def venues_to_dataframe(raw_json):
    wanted = pd.DataFrame()
    if raw_json != None:
        nearby_venues = json_normalize(raw_json['Venues'])
        wanted = nearby_venues[['id','name','categories','location.lat','location.lng','location.distance']]
        wanted.insert(wanted.shape[1],'category',wanted['categories'].apply(get_primary_category))
        for k in [ 'Borough', 'Postcode']: # Insert in back order
            wanted.insert(0, k, raw_json[k])

    wanted.columns = ['Postcode', 'Borough', 'VenueId','name','categories','latitude','longitude','distance','category']
    return wanted

In [ ]:
# Transformation function
# Given a postal_code row convert to a list of venue ids and category
def apply_get_venues(row):
    #print(row['Postcode'])
    return get_venues_for(
                    row['Postcode']     
                    ,row['Borough']
                    ,row['Latitude']
                    ,row['Longitude']
                  )

### 4. Get all the venues and associate categories

In [ ]:
# Run through all the postal codes and build up array of results
processed = 0
rawVenues = []

for row in postal_codes_final.sort_values(['Latitude','Longitude']).iterrows():
    raw = get_venues_for(row[0],*row[1][['Borough','Latitude','Longitude']].values)
    if raw != None:    
        #print(raw[['id','location.lat','location.lng','name']].sort_values(['id']))
        rawVenues.append(raw)

    #if processed == 0: print(raw[0])
    processed = processed + 1
print('Done Raw Results: ', processed)
print('len(rawVenues)',len(rawVenues))

### 5.  Save results into flat file to save time when rerunning.

In [ ]:
# Take the data and save to file
with open('venue-data2.json', 'w', encoding='utf-8') as f:
    json.dump(rawVenues, f, ensure_ascii=False, indent=4)

### 6. Now load from flatfile

In [ ]:
with open('venue-data2.json', 'r', encoding='utf-8') as f:
    rawVenues = json.load(f)


### 7. Extract values from raw json

In [ ]:
# Run through all the postal codes and build up array of results
processed = 0
todo = []

for row in rawVenues:
    raw = venues_to_dataframe(row)
    todo.append(raw)
    processed = processed + 1
print('Done Venues Results: ', processed)
venues = pd.concat(todo,axis='index')
venues.drop(columns=['categories'],inplace=True)


In [ ]:
keycolumns=['Postcode','Borough']
onehot = pd.get_dummies(venues[['category']], prefix="", prefix_sep="")
work = pd.concat([venues,onehot], axis=1)
mi = pd.MultiIndex.from_frame(work[keycolumns])
work.set_index(mi, inplace=True)
work.drop(columns=keycolumns,inplace=True)
work.groupby('Postcode').mean()
print(len(venues['VenueId']),len(venues['VenueId'].unique()))

In [ ]:
len(venues['category'].unique())

In [ ]:
q=geodata.iloc[0]
q=[q['Latitude'],q['Longitude']]
print(type(q))#,q['Latitude'],q['Longitude'])
folMap = folium.Map(location=q,zoom_start=9)
mc = folium.plugins.MarkerCluster()
for loc in geodata.iterrows():
    data = loc[1]
    folium.CircleMarker([data['Latitude'],data['Longitude']], radius=10, popup=data['Postal Code']).add_to(folMap)

count=0
for loc in venues.iterrows():
    data=loc[1]
    # folium display of map fails if there are too many markers.
    # Using a cluster allows us to increase number, but it still fails when over 9K
    # "Randomly" pick 1 out of 6 locations to map
    if count % 6 == 0:
        folium.Marker([data['latitude'],data['longitude']], popup=data['name']).add_to(mc)
    count = count+1
    
folMap.add_child(mc)
folMap